In [1]:
#n.p check input at native or standard, check mask as native or standard, check time series, check parameters too of course

In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image
from nilearn.glm.first_level import compute_regressor
import logging
from brainiak.searchlight.searchlight import Searchlight, Ball
from statsmodels.tsa.stattools import grangercausalitytests
import sys
import gc
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Scramble'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
subs = ['sub-025']  # For testing, you can include specific subjects

run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# Searchlight parameters
searchlight_radius = 2  # in voxels, adjust as needed
max_blk_edge = 10
pool_size = 1

# Constants
VOL_PER_RUN = 184
TR = 2.0
first_fix = 8  # Number of initial volumes to skip

# Load whole brain mask
whole_brain_mask = image.load_img('/user_data/csimmon2/git_repos/ptoc/roiParcels/mruczek_parcels/binary/all_visual_areas.nii.gz')

def load_data(sub, run_combo, raw_dir, whole_brain_mask, first_fix):
    all_runs = []
    for run in run_combo:
        curr_run = image.load_img(f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/registered_data/filtered_func_data_standard.nii.gz')
        curr_run = image.get_data(image.clean_img(curr_run, standardize=True, mask_img=whole_brain_mask))
        curr_run = curr_run[:,:,:,first_fix:]
        all_runs.append(curr_run)
    
    bold_vol = np.concatenate(np.array(all_runs), axis=3)
    return bold_vol

def make_psy_cov(runs, ss):
    """
    Create psychological covariate data for the specified runs and subject.
    """
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    total_vols = VOL_PER_RUN * len(runs)
    times = np.arange(0, total_vols * TR, TR)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            return np.zeros((total_vols, 1))  # Return a zeros array if file not found

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * VOL_PER_RUN * TR
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy

def extract_pips_timeseries(bold_vol, pips_mask):
    reshaped_mask = np.reshape(pips_mask, (91, 109, 91, 1))
    masked_img = reshaped_mask * bold_vol
    pips_ts = masked_img.reshape(-1, bold_vol.shape[3])
    pips_ts = pips_ts[~np.all(pips_ts == 0, axis=1)]
    return np.mean(pips_ts, axis=0)

def gca_measure(data, mask, myrad, bcvar):
    pips_ts, psy = bcvar
    
    # Apply mask to get the searchlight sphere time series
    sphere_ts = data[0][mask].mean(axis=0)
    
    # Combine time series into a single 2D array
    combined_ts = np.column_stack((sphere_ts, pips_ts, psy))
    
    try:
        gc_pips_to_sphere = grangercausalitytests(combined_ts[:, [1, 0, 2]], maxlag=1, verbose=False)
        gc_sphere_to_pips = grangercausalitytests(combined_ts[:, [0, 1, 2]], maxlag=1, verbose=False)
        
        # Calculate the difference in F-statistics
        f_diff = gc_pips_to_sphere[1][0]['ssr_ftest'][0] - gc_sphere_to_pips[1][0]['ssr_ftest'][0]
    except Exception as e:
        print(f"Error in grangercausalitytests: {str(e)}")
        return np.nan
    
    return f_diff

def run_searchlight(fmri_data, whole_brain_mask, pips_mask, psy):
    # Extract pIPS time series
    pips_ts = extract_pips_timeseries(fmri_data, pips_mask)
    
    # Setup searchlight
    sl = Searchlight(sl_rad=searchlight_radius, max_blk_edge=max_blk_edge, shape=Ball)
    
    # Distribute data and broadcast pips_ts and psy
    sl.distribute([fmri_data], whole_brain_mask)
    sl.broadcast((pips_ts, psy))
    
    # Run searchlight
    sl_result = sl.run_searchlight(gca_measure, pool_size=pool_size)
    
    return sl_result

# Main execution
if __name__ == "__main__":
    for sub in subs:
        for run_combo in run_combos:
            try:
                logging.info(f"Processing subject {sub}, runs {run_combo}")
                
                # Load fMRI data
                fmri_data = load_data(sub, run_combo, raw_dir, whole_brain_mask, first_fix)
                
                # Load pIPS mask
                pips_mask_file = f'/user_data/csimmon2/git_repos/ptoc/roiParcels/pIPS.nii.gz'
                pips_mask = nib.load(pips_mask_file).get_fdata().astype(bool)
                
                # Create psychological covariate
                psy = make_psy_cov(run_combo, sub)
                
                logging.info(f"Data shapes - fMRI: {fmri_data.shape}, Whole Brain Mask: {whole_brain_mask.get_fdata().shape}, pIPS Mask: {pips_mask.shape}, PSY: {psy.shape}")
                
                # Run searchlight
                sl_result = run_searchlight(fmri_data, whole_brain_mask.get_fdata(), pips_mask, psy)
                
                # Save results
                result_file = f'{results_dir}/searchlight_gca_{sub}_runs{"_".join(map(str, run_combo))}.nii.gz'
                affine = nib.load(f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-01/registered_data/filtered_func_data_standard.nii.gz').affine
                nib.save(nib.Nifti1Image(sl_result, affine), result_file)
                
                logging.info(f"Completed analysis for subject {sub}, runs {run_combo}")
                
            except Exception as e:
                logging.error(f"Error processing subject {sub}, runs {run_combo}: {str(e)}")
            finally:
                gc.collect()

2024-10-08 10:53:10,587 - INFO - Processing subject sub-025, runs [1, 2]
/home/csimmon2/anaconda3/envs/brainiak_env/lib/python3.7/site-packages/nilearn/image/image.py:1282: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  **clean_kwargs,
